In [50]:
#Load saved model

from model import ner_bio
import torch
import pandas as pd
import torch.nn as nn
from pytorch_pretrained_bert import BertModel,BertConfig, BertForPreTraining
import tensorflow as tf
import re
import torch
import numpy as np
from tqdm import tqdm, trange
import os
import csv
from pytorch_pretrained_bert import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn as nn
MAX_LEN = 75
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer(vocab_file='biobert_v1.0_pubmed_pmc/vocab.txt', do_lower_case=False)

In [11]:
def sentence_retriver(path):
    
    with open(path) as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        sentences = []
        tags = []
        sent = []
        tag = []
        for row in reader:
            if len(row) == 0:
                if len(sent) != len(tag):
                    print('Error')
                    break
                sentences.append(sent)
                tags.append(tag)
                sent = []
                tag = []
            else:
                sent.append(row[0])
                tag.append(row[1])
            
    return sentences, tags

def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenizing the words
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels


In [8]:
data = pd.read_csv('tags_small.csv')
tag_values = data['tags'].values
vocab_len = len(tag_values)

In [9]:
data.head()
# len(tag_values)

,tags
0,I-Cellular_component
1,E-Gene_or_gene_product
2,I-Organism_subdivision
3,I-Organism_substance
4,B-Gene_or_gene_product


In [102]:
# Test model
rootdir = './BioNLP'
import os
import csv
sentences = []
tags = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file == 'test.tsv':
#             print(os.path.join(subdir, file))
            path_ = os.path.join(subdir, file)
            sent, tag =sentence_retriver(path_)
            sentences.extend(sent)
            tags.extend(tag)
            
sentences = sentences[0:5000]
tags = tags[0:5000]

tag_list = []
for idx,tg in enumerate(tags):
    for t in tg:
        if t != 'O':
            tag_list.append(idx)
        break
        
test_sentences = []
test_labels =[]

In [106]:
import random

test_idx = random.choice(tag_list)

# test_idx = 9999
test_sentence = sentences[test_idx]
test_label = tags[test_idx]

test_sentences.append(test_sentence)
test_labels.append(test_labels)
for l,s in zip(test_sentence,test_label):
    print(l,'--',s,'\n')


Hfq -- B-Protein 

was -- O 

observed -- O 

to -- O 

modulate -- O 

numerous -- O 

proteins -- O 

involved -- O 

in -- O 

envelope -- O 

stress -- O 

under -- O 

AMM -- O 

- -- O 

2 -- O 

conditions -- O 

, -- O 

including -- O 

FkpA -- B-Protein 

, -- O 

SurA -- B-Protein 

, -- O 

HtrA -- B-Protein 

, -- O 

NlpB -- B-Protein 

, -- O 

NmpC -- B-Protein 

, -- O 

ClpA -- B-Protein 

, -- O 

SlyD -- B-Protein 

, -- O 

RseA -- B-Protein 

, -- O 

RseB -- B-Protein 

, -- O 

RpoE -- B-Protein 

, -- O 

OmpF -- B-Protein 

, -- O 

and -- O 

HtpG -- B-Protein 

, -- O 

which -- O 

were -- O 

all -- O 

up -- O 

- -- O 

regulated -- O 

in -- O 

the -- O 

Deltahfq -- B-Protein 

mutant -- O 

strain -- O 

. -- O 



In [107]:
# print(len(test_sentences),len(test_labels))
i=-1
print(" ".join(test_sentences[i]),'\n')
print(test_labels[i])

Hfq was observed to modulate numerous proteins involved in envelope stress under AMM - 2 conditions , including FkpA , SurA , HtrA , NlpB , NmpC , ClpA , SlyD , RseA , RseB , RpoE , OmpF , and HtpG , which were all up - regulated in the Deltahfq mutant strain . 

[[...], [...], [...], [...]]


In [25]:
text = """Regulation of Saccharomyces cerevisiae kinetochores by the type 1 phosphatase Glc7p."""

In [26]:
import nltk
sent_text = nltk.sent_tokenize(text)

In [27]:
sent_text

['Regulation of Saccharomyces cerevisiae kinetochores by the type 1 phosphatase Glc7p.']

In [30]:
for sentence in sent_text:
    tokenized_text = nltk.word_tokenize(sentence)

In [31]:
tokenized_text

['Regulation',
 'of',
 'Saccharomyces',
 'cerevisiae',
 'kinetochores',
 'by',
 'the',
 'type',
 '1',
 'phosphatase',
 'Glc7p',
 '.']

In [76]:
text = """Directed cell migration is critical across biological processes spanning healing to cancer invasion, yet no existing tools allow real-time interactive guidance over such migration. We present a new bioreactor that harnesses electrotaxis—directed cell migration along electric field gradients—by integrating four independent electrodes under computer control to dynamically program electric field patterns, and hence steer cell migration. Using this platform, we programmed and characterized multiple precise, two-dimensional collective migration maneuvers in renal epithelia and primary skin keratinocyte ensembles. First, we demonstrated on-demand, 90-degree collective turning. Next, we developed a universal electrical stimulation scheme capable of programming arbitrary 2D migration maneuvers such as precise angular turns and migration in a complete circle. Our stimulation scheme proves that cells effectively time-average electric field cues, helping to elucidate the transduction timescales in electrotaxis. Together, this work represents an enabling platform for controlling cell migration with broad utility across many cell types."""

In [77]:
sent_text = nltk.sent_tokenize(text)
tokenized_text = []
for sentence in sent_text:
    tokenized_text.append(nltk.word_tokenize(sentence))

In [78]:
len(tokenized_text[0])

26

In [79]:
print(tokenized_text)

[['Directed', 'cell', 'migration', 'is', 'critical', 'across', 'biological', 'processes', 'spanning', 'healing', 'to', 'cancer', 'invasion', ',', 'yet', 'no', 'existing', 'tools', 'allow', 'real-time', 'interactive', 'guidance', 'over', 'such', 'migration', '.'], ['We', 'present', 'a', 'new', 'bioreactor', 'that', 'harnesses', 'electrotaxis—directed', 'cell', 'migration', 'along', 'electric', 'field', 'gradients—by', 'integrating', 'four', 'independent', 'electrodes', 'under', 'computer', 'control', 'to', 'dynamically', 'program', 'electric', 'field', 'patterns', ',', 'and', 'hence', 'steer', 'cell', 'migration', '.'], ['Using', 'this', 'platform', ',', 'we', 'programmed', 'and', 'characterized', 'multiple', 'precise', ',', 'two-dimensional', 'collective', 'migration', 'maneuvers', 'in', 'renal', 'epithelia', 'and', 'primary', 'skin', 'keratinocyte', 'ensembles', '.'], ['First', ',', 'we', 'demonstrated', 'on-demand', ',', '90-degree', 'collective', 'turning', '.'], ['Next', ',', 'we',

In [74]:
def tokenize_and_preserve(sentence):
    tokenized_sentence = []
    
    for word in sentence:
        # Tokenizing the words
        tokenized_word = tokenizer.tokenize(word)   
        tokenized_sentence.extend(tokenized_word)

    return tokenized_sentence


In [80]:
tokenized_texts_ = [
    tokenize_and_preserve(sent)
    for sent in tokenized_text
]

In [83]:
input_ids = [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_]
                        

In [84]:
input_ids

[[16698,
  2765,
  10348,
  1110,
  3607,
  1506,
  7269,
  5669,
  14827,
  9523,
  1106,
  4182,
  4923,
  117,
  1870,
  1185,
  3685,
  5537,
  2621,
  1842,
  118,
  1159,
  12196,
  8815,
  1166,
  1216,
  10348,
  119],
 [1284,
  1675,
  170,
  1207,
  25128,
  11811,
  9363,
  1115,
  20211,
  1279,
  24266,
  1777,
  8745,
  1116,
  783,
  2002,
  2765,
  10348,
  1373,
  3651,
  1768,
  19848,
  1116,
  783,
  1118,
  26975,
  1300,
  2457,
  24266,
  4704,
  1223,
  2775,
  1654,
  1106,
  9652,
  2716,
  1788,
  3651,
  1768,
  6692,
  117,
  1105,
  7544,
  25284,
  2765,
  10348,
  119],
 [7993,
  1142,
  3482,
  117,
  1195,
  18693,
  1105,
  6858,
  2967,
  10515,
  117,
  1160,
  118,
  8611,
  7764,
  10348,
  27559,
  1107,
  1231,
  7050,
  174,
  18965,
  18809,
  1465,
  1105,
  2425,
  2241,
  180,
  5970,
  20064,
  3457,
  1566,
  24957,
  119],
 [1752,
  117,
  1195,
  7160,
  1113,
  118,
  4555,
  117,
  3078,
  118,
  2178,
  7764,
  3219,
  119],
 [5893,


In [87]:
tokens = tokenizer.convert_ids_to_tokens(input_ids[1])
new_tokens, new_labels = [], []
for token in tokens:
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        
        new_tokens.append(token)

In [88]:
new_tokens

['We',
 'present',
 'a',
 'new',
 'bioreactor',
 'that',
 'harnesses',
 'electrotaxis',
 '—',
 'directed',
 'cell',
 'migration',
 'along',
 'electric',
 'field',
 'gradients',
 '—',
 'by',
 'integrating',
 'four',
 'independent',
 'electrodes',
 'under',
 'computer',
 'control',
 'to',
 'dynamically',
 'program',
 'electric',
 'field',
 'patterns',
 ',',
 'and',
 'hence',
 'steer',
 'cell',
 'migration',
 '.']

In [58]:
print(input_ids[0])

[[141, 178, 187, 174, 172, 189, 174, 173], [172, 174, 181, 181], [182, 178, 176, 187, 170, 189, 178, 184, 183], [178, 188], [172, 187, 178, 189, 178, 172, 170, 181], [170, 172, 187, 184, 188, 188], [171, 178, 184, 181, 184, 176, 178, 172, 170, 181], [185, 187, 184, 172, 174, 188, 188, 174, 188], [188, 185, 170, 183, 183, 178, 183, 176], [177, 174, 170, 181, 178, 183, 176], [189, 184], [172, 170, 183, 172, 174, 187], [178, 183, 191, 170, 188, 178, 184, 183], [117], [194, 174, 189], [183, 184], [174, 193, 178, 188, 189, 178, 183, 176], [189, 184, 184, 181, 188], [170, 181, 181, 184, 192], [187, 174, 170, 181, 118, 189, 178, 182, 174], [178, 183, 189, 174, 187, 170, 172, 189, 178, 191, 174], [176, 190, 178, 173, 170, 183, 172, 174], [184, 191, 174, 187], [188, 190, 172, 177], [182, 178, 176, 187, 170, 189, 178, 184, 183], [119]]


In [52]:
for txt in tokenized_text:
    print(txt)

['Directed', 'cell', 'migration', 'is', 'critical', 'across', 'biological', 'processes', 'spanning', 'healing', 'to', 'cancer', 'invasion', ',', 'yet', 'no', 'existing', 'tools', 'allow', 'real-time', 'interactive', 'guidance', 'over', 'such', 'migration', '.']
['We', 'present', 'a', 'new', 'bioreactor', 'that', 'harnesses', 'electrotaxis—directed', 'cell', 'migration', 'along', 'electric', 'field', 'gradients—by', 'integrating', 'four', 'independent', 'electrodes', 'under', 'computer', 'control', 'to', 'dynamically', 'program', 'electric', 'field', 'patterns', ',', 'and', 'hence', 'steer', 'cell', 'migration', '.']
['Using', 'this', 'platform', ',', 'we', 'programmed', 'and', 'characterized', 'multiple', 'precise', ',', 'two-dimensional', 'collective', 'migration', 'maneuvers', 'in', 'renal', 'epithelia', 'and', 'primary', 'skin', 'keratinocyte', 'ensembles', '.']
['First', ',', 'we', 'demonstrated', 'on-demand', ',', '90-degree', 'collective', 'turning', '.']
['Next', ',', 'we', 'dev

In [24]:
tokenized_sentence = []
input_ids = []
for sent in text_split:
    tokenized_sentence.extend(tokenizer.tokenize(sent))
    
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(tokenized_sentence) ],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

input_ids = torch.tensor(input_ids).cuda()
attention_masks = torch.tensor(attention_masks).cuda()

In [29]:
config = BertConfig.from_json_file('biobert_v1.0_pubmed_pmc/bert_config.json')
vocab_len = 65
model = ner_bio(vocab_len,config,state_dict=None)
model.load_state_dict(torch.load('app/BIONER_classifier.pt', map_location=device))

In [1]:
import nltk

text = """Directed cell migration is critical across biological processes spanning healing to cancer invasion, yet no existing tools allow real-time interactive guidance over such migration. We present a new bioreactor that harnesses electrotaxis—directed cell migration along electric field gradients—by integrating four independent electrodes under computer control to dynamically program electric field patterns, and hence steer cell migration. Using this platform, we programmed and characterized multiple precise, two-dimensional collective migration maneuvers in renal epithelia and primary skin keratinocyte ensembles. First, we demonstrated on-demand, 90-degree collective turning. Next, we developed a universal electrical stimulation scheme capable of programming arbitrary 2D migration maneuvers such as precise angular turns and migration in a complete circle. Our stimulation scheme proves that cells effectively time-average electric field cues, helping to elucidate the transduction timescales in electrotaxis. Together, this work represents an enabling platform for controlling cell migration with broad utility across many cell types."""

In [2]:
sent_text = nltk.sent_tokenize(text)

<All keys matched successfully>

In [39]:
model.cuda()
model.eval()
with torch.no_grad():
    _, y_hat = model(input_ids,attention_mask=attention_masks)
label_indices = y_hat.to('cpu').numpy()

In [40]:
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)

In [41]:
for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))

B-Immaterial_anatomical_entity	Regulation
B-Immaterial_anatomical_entity	of
B-Immaterial_anatomical_entity	Saccharomyces
B-Immaterial_anatomical_entity	cerevisiae
E-Amino_acid	kinetochores
B-Immaterial_anatomical_entity	by
B-Immaterial_anatomical_entity	the
B-Immaterial_anatomical_entity	type
B-Immaterial_anatomical_entity	1
B-Immaterial_anatomical_entity	phosphatase
B-Immaterial_anatomical_entity	Glc7p
B-Immaterial_anatomical_entity	.
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_system	[PAD]
B-Anatomical_s